# Trend Analysis
## Refactored to use the new Modular Architecture

In [8]:
%load_ext autoreload
%autoreload 2
import sys
import os

# import from rootdir
sys.path.append(os.path.abspath(".."))

import pandas as pd
import matplotlib.pyplot as plt

from core.data import ResearchBitcoinLoader
from indicators.trends import AdaptiveTrend
from evaluation.labelers import ZigZagLabeler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load Data

In [9]:
# Using the Cache file directly if available, or API
try:
    df = pd.read_csv("resources/BitCoinLab/net_unrealized_profit_loss_2024-11-22---2025-11-21.csv")
    df["date"] = pd.to_datetime(df["date"])
    df.set_index("date", inplace=True)
except FileNotFoundError:
    print("Local file not found, trying API...")
    loader = ResearchBitcoinLoader(token="850d3cf9-804d-460b-8cdd-4f09983de4c2", item="net_unrealized_profit_loss/net_unrealized_profit_loss")
    df = loader.load()

print(df.head())

Local file not found, trying API...
Fetching from https://api.researchbitcoin.net/v1/net_unrealized_profit_loss/net_unrealized_profit_loss...


OSError: Cannot save file into a non-existent directory: 'ohlcv'

### Adaptive Trend Analysis (segmented mean)

In [6]:
# The new class replaces 'sliding_window_analysis'
trend_algo = AdaptiveTrend(initial_window=5, stability_threshold=0.007, column="net_unrealized_profit_loss")
trend_data = trend_algo.calculate(df)

# Merge for plotting
df_trend = df.join(trend_data)

# Plot
plt.figure(figsize=(14, 7))
plt.plot(df_trend.index, df_trend["net_unrealized_profit_loss"], label="Signal", color="gray", alpha=0.4)
plt.plot(df_trend.index, df_trend["adaptive_mean"], label="Adaptive Mean", color="red", linewidth=2)

# Highlight Change Points (jumps in mean)
changes = df_trend["adaptive_mean"].diff().abs() > 1e-6
change_points = df_trend[changes]
plt.scatter(change_points.index, change_points["adaptive_mean"], color="blue", s=30, label="Change Points")

plt.title("Segmented Mean Plot (Adaptive Trend)")
plt.legend()
plt.show()

NameError: name 'df' is not defined

### ZigZag Ground Truth Labeling

In [7]:
# The new class replaces 'zigzag_algorithm'
zigzag = ZigZagLabeler(deviation_pct=0.1, column="net_unrealized_profit_loss")
df_labeled = zigzag.label(df)

# Plot
plt.figure(figsize=(14, 7))
plt.plot(df_labeled.index, df_labeled["net_unrealized_profit_loss"], label="NUPL", color="gray", alpha=0.6)

# Plot the ZigZag Line
zz_line = df_labeled[df_labeled["zigzag_line"].notna()]
plt.plot(zz_line.index, zz_line["zigzag_line"], color="red", marker="o", markersize=3, label="ZigZag Path")

# Plot Peaks and Troughs
peaks = df_labeled[df_labeled["is_peak"]]
troughs = df_labeled[df_labeled["is_trough"]]

plt.scatter(peaks.index, peaks["net_unrealized_profit_loss"], marker="^", color="red", s=100, label="Peaks", zorder=5)
plt.scatter(troughs.index, troughs["net_unrealized_profit_loss"], marker="v", color="green", s=100, label="Troughs", zorder=5)

plt.title("ZigZag Algorithm for Peaks and Troughs")
plt.legend()
plt.grid(True, linestyle=":", alpha=0.7)
plt.show()

NameError: name 'df' is not defined